In [ ]:
!nvidia-smi

Thu Aug 25 05:50:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import numpy as np
import pandas as pd
import timeit

In [ ]:
# a very simple arithmetic based method

def sum_nums(a, b):
    return a + b

In [ ]:
df = pd.DataFrame({'series1':np.random.randn(1000000), 'series2':np.random.randn(1000000)})

In [ ]:
df

,series1,series2
0,-0.097334,-0.628245
1,-0.952810,-0.606038
2,0.492993,-0.542700
3,0.862114,-0.573252
4,1.504483,-0.664344
...,...,...
999995,-0.497209,-1.271519
999996,-0.168351,0.113553
999997,2.295218,0.574387
999998,0.090730,0.472631


## Check to see if all methods result in the same output

In [ ]:
series3 = df.apply(lambda df: sum_nums(df['series1'],df['series2']),axis=1)
series3[0:5].to_list(), series3[-5:].to_list()

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

In [ ]:
series3 = [sum_nums(a, b) for a, b in df.itertuples(index=False)]
series3[0:5], series3[-5:]

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

In [ ]:
series3 = [sum_nums(a, b) for a, b in zip(df['series1'],df['series2'])]
series3[0:5], series3[-5:]

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

In [ ]:
series3 = np.vectorize(sum_nums)(df['series1'],df['series2'])
series3[0:5].tolist(), series3[-5:].tolist()

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

In [ ]:
series3 = df['series1'] + df['series2']
series3[0:5].tolist(), series3[-5:].tolist()

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

In [ ]:
series3 = np.add(df['series1'].to_numpy(),df['series2'].to_numpy())
series3[0:5].tolist(), series3[-5:].tolist()

([-0.7255789514925267,
  -1.558847935596728,
  -0.04970686690096088,
  0.28886155115905787,
  0.8401395883248813],
 [-1.7687276854821357,
  -0.05479793332391117,
  2.8696052118064332,
  0.5633608662624958,
  1.8635641046562685])

## Timit setup

In [ ]:
setup = '''
import numpy as np
import pandas as pd
series = np.array([np.arange(1,1000000),np.arange(2,1000001)]).T
df = pd.DataFrame(series, columns=['series1','series2'])
def sum_nums(a, b):
    return a + b
'''

In [ ]:
apply_func = '''
series3 = df.apply(lambda df: sum_nums(df['series1'],df['series2']),axis=1)
'''

In [ ]:
iter_tup_func = '''
series3 = [sum_nums(a, b) for a, b in df.itertuples(index=False)]
'''

In [ ]:
list_comp_func = '''
series3 = [sum_nums(a, b) for a, b in zip(df['series1'],df['series2'])]
'''

In [ ]:
vectorize_func = '''
series3 = np.vectorize(sum_nums)(df['series1'],df['series2'])
'''

In [ ]:
df_func = '''
series3 = df['series1'] + df['series2']
'''

In [ ]:
numpy_func = '''
series3 = np.add(df['series1'].to_numpy(),df['series2'].to_numpy())
'''

## Run timit for different methods

In [ ]:
timeit.timeit(stmt=apply_func, setup=setup, number=1)

11.60054962800001

In [ ]:
timeit.timeit(stmt=iter_tup_func, setup=setup, number=10)

6.1248978530000215

In [ ]:
timeit.timeit(stmt=list_comp_func, setup=setup, number=100)

29.314812690999986

In [ ]:
timeit.timeit(stmt=vectorize_func, setup=setup, number=100)

22.125779193

In [ ]:
timeit.timeit(stmt=df_func, setup=setup, number=1000)

1.66047883600001

In [ ]:
timeit.timeit(stmt=numpy_func, setup=setup, number=1000)

1.421019316000013

# A more complicated example

You could of couse put logic into the function. However, this will negate the use of both a simple numpy operation, and a simple pandas operation as detailed in the section above.

This section will therefore consider the potential to speed up conditional methods.

In [ ]:
def categorise(a, b):
    if a < 0:
        return a * 2 + b
    elif b < 0:
        return a + 2 * b
    else:
        return None

## Check to see if all methods result in the same output

In [ ]:
series3 = df.apply(lambda df: categorise(df['series1'],df['series2']),axis=1)
series3[0:5].to_list(), series3[-5:].to_list()

([-0.8229132567991985,
  -2.511657437159238,
  -0.5924064086399732,
  -0.28439056906206195,
  0.17579605620671224],
 [-2.265936193593988, -0.2231485315173992, nan, nan, 1.3878564038272145])

In [ ]:
series3 = [categorise(a, b) for a, b in df.itertuples(index=False)]
series3[0:5], series3[-5:]

([-0.8229132567991985,
  -2.511657437159238,
  -0.5924064086399732,
  -0.28439056906206195,
  0.17579605620671224],
 [-2.265936193593988, -0.2231485315173992, None, None, 1.3878564038272145])

In [ ]:
series3 = [categorise(a, b) for a, b in zip(df['series1'],df['series2'])]
series3[0:5], series3[-5:]

([-0.8229132567991985,
  -2.511657437159238,
  -0.5924064086399732,
  -0.28439056906206195,
  0.17579605620671224],
 [-2.265936193593988, -0.2231485315173992, None, None, 1.3878564038272145])

In [ ]:
series3 = np.vectorize(categorise)(df['series1'],df['series2'])
series3[0:5].tolist(), series3[-5:].tolist()

([-0.8229132567991985,
  -2.511657437159238,
  -0.5924064086399732,
  -0.28439056906206195,
  0.17579605620671224],
 [-2.265936193593988, -0.2231485315173992, nan, nan, 1.3878564038272145])

## Timeit setup

In [ ]:
setup = '''
import numpy as np
import pandas as pd
series = np.array([np.arange(1,1000000),np.arange(2,1000001)]).T
df = pd.DataFrame(series, columns=['series1','series2'])
def categorise(a, b):
    if a < 0:
        return a * 2 + b
    elif b < 0:
        return a + 2 * b
    else:
        return None
'''

In [ ]:
apply_func = '''
series3 = df.apply(lambda df: categorise(df['series1'],df['series2']),axis=1)
'''

In [ ]:
iter_tup_func = '''
series3 = [categorise(a, b) for a, b in df.itertuples(index=False)]
'''

In [ ]:
list_comp_func = '''
series3 = [categorise(a, b) for a, b in zip(df['series1'],df['series2'])]
'''

In [ ]:
vectorize_func = '''
series3 = np.vectorize(categorise)(df['series1'],df['series2'])
'''

## Run timit for different methods

In [ ]:
timeit.timeit(stmt=apply_func, setup=setup, number=1)

11.575162051999996

In [ ]:
timeit.timeit(stmt=iter_tup_func, setup=setup, number=10)

6.410262793000015

In [ ]:
timeit.timeit(stmt=list_comp_func, setup=setup, number=100)

31.12600953399999

In [ ]:
timeit.timeit(stmt=vectorize_func, setup=setup, number=100)

21.41128515600002